# BatchNorm

批量归一化的本质目的：学习底部层的时候避免顶部层变化                     
所以只有在使用深层的网络结构的时候才会使用BN，浅层的如mlp效果不明显                                      
优点：不会改变模型的精度，但是可以加速模型收敛                                             
首先我们先明确引起变化的原因，是每一层的方差和均值的分布不同。所以我们要固定所有层的分布，尽量符合同一个分布。              
批量归一化的作用原理：固定小批量里的均值和方差     


优点：不会改变模型的精度，但是可以加速模型收敛    

$\mu_B=\frac{1}{|B|} \sum_{i \in B} x_i \text { and } \sigma_B^2=\frac{1}{|B|} \sum_{i \in B}\left(x_i-\mu_B\right)^2+\epsilon$


1. 首先求出均值和方差
2. 再做额外的调整，输入的是实际数据$x_i$，输出是归一化后的$x_{i+1}$，其中$gama$和$beta$是可以学习的参数


$y=\frac{x-\mathrm{E}[x]}{\sqrt{\operatorname{Var}[x]+\epsilon}} * \gamma+\beta$

In [2]:
import torch
import torch.nn as nn

In [36]:
def batch_norm(X:torch.Tensor, 
               gamma:torch.Tensor, 
               beta:torch.Tensor, 
               moving_mean:torch.Tensor, 
               moving_var:torch.Tensor, 
               eps, 
               momentum):
    if not torch.is_grad_enabled():
        # 通过is_grad_enabled来判断当前模式是训练模式还是预测模式
        x_hat = (X-moving_mean)/torch.sqrt(moving_var+eps)
    
    else:
        assert len(X.shape) in (2, 4)
        #判断是全连接层还是卷积层，2代表全连接层，样本数和特征数；4代表卷积层，批量数，通道数，高宽
        if len(X.shape) == 2:
            mean = X.mean(dim=0, keepdim=True) # []
            var = ((X-mean)**2).mean(dim=0, keepdim=True)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。
            # 这里我们需要保持X的形状以便后面可以做广播运算
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            #1*n*高*宽
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # 训练模式下，用当前的均值和方差做标准化
        x_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * x_hat + beta  # 缩放和移位
    return Y, moving_mean.data, moving_var.data


class BatchNorm(nn.Module):
    def __init__(self, channels, num_dims) -> None:
        # num_features：完全连接层的输出数量或卷积层的输出通道数。
        # num_dims：2表示完全连接层，4表示卷积层
        super().__init__()
        self.channels = channels
        if num_dims==2:
            shape = (1, channels)
        else:
            shape = (1, channels, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)
        
    def forward(self, X:torch.Tensor):
        # 如果X不在内存上，将moving_mean和moving_var
        # 复制到X所在显存上
        if X.device != self.moving_mean.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var
        Y, self.moving_mean, self.moving_var = batch_norm(X, 
                                                          self.gamma, 
                                                          self.beta, 
                                                          self.moving_mean,
                                                          self.moving_var,
                                                          eps=1e-5, 
                                                          momentum=0.9)
        return Y

In [37]:
model = BatchNorm(3, num_dims=4)
model2 = nn.BatchNorm2d(3)
input_ = torch.randn(size=(5, 3, 24, 24))
res = model(input_)
res2 = model2(input_)
p = torch.allclose(res, res2)
p

True